In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = np.load('../AC.npz', allow_pickle=True)
F = data['F']
time = data['time']
speed = data['speed']
n_F = len(F)
n_generators = speed[0].shape[1]

In [ ]:

n = speed[0].shape[1]
cmap = plt.get_cmap('viridis', n)
fig,ax = plt.subplots(n_F, 1, figsize=(8,1.5*n_F))
for i,(f,t,spd) in enumerate(zip(F, time, speed)):
    idx = t > t[-1] - 10 / f
    for j in range(n):
        ax[i].plot(t[idx], spd[idx,j] - 1, lw=1, label=f'Gen. {j+1}')
    ax[i].set_ylabel('Speed - 1 [p.u.]')
    ax[i].set_title(f'F = {f} Hz')
ax[-1].set_xlabel('Time [s]')
ax[0].legend(loc='lower left')
sns.despine()
fig.tight_layout()

In [ ]:
μ = 1
gamma_c = np.zeros((n_F, n_generators))
gamma_s = np.zeros((n_F, n_generators))
for i,(f,t,spd) in enumerate(zip(F, time, speed)):
    dt = t[1] - t[0]
    idx = t > t[-1] - μ / f
    for j in range(n_generators):
        gamma_c[i,j] = f/μ*dt*np.cos(2*np.pi*f*t[np.newaxis,idx]) @ (spd[idx,j]-1)
        gamma_s[i,j] = f/μ*dt*np.sin(2*np.pi*f*t[np.newaxis,idx]) @ (spd[idx,j]-1)

In [ ]:
fig,ax = plt.subplots(1, 1, sharex=True, figsize=(8,4))
for i in range(n_generators):
    ax.plot(F, 10*np.log10(gamma_c[:,i]**2 + gamma_s[:,i]**2), label=f'Gen {i+1}')
ax.set_xscale('log')
ax.legend(loc='lower left', frameon=False)
ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel('dB 10')
sns.despine()